In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import math


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/'My Drive'/doan

/content/drive/My Drive/doan


**IMPORT DATA**

In [4]:
image_size = (64, 64)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Class_Human",
    validation_split=0.2,
    subset="training",
    seed=18,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Class_Human",
    validation_split=0.2,
    subset="validation",
    seed=18,
    image_size=image_size,
    batch_size=batch_size,
)


Found 21758 files belonging to 2 classes.
Using 17407 files for training.
Found 21758 files belonging to 2 classes.
Using 4351 files for validation.


In [5]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

**TRAINNING**

Model

In [6]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,concatenate,AveragePooling2D,Rescaling
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from keras.models import Sequential

In [7]:
def inception_layer(prev_layer, filters_1_1x1=16, filters_2_1x1=24, filters_2_3x3=32, filters_3_1x1=4, filters_3_5x5=8, filters_4_1x1=8, name=None):
  conv1 = Conv2D(filters=filters_1_1x1, kernel_size=(1,1), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(prev_layer)

  conv2 = Conv2D(filters=filters_2_1x1, kernel_size=(1,1), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(prev_layer)
  conv2 = Conv2D(filters=filters_2_3x3, kernel_size=(3,3), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(conv2)

  conv3 = Conv2D(filters=filters_3_1x1, kernel_size=(1,1), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(prev_layer)
  conv3 = Conv2D(filters=filters_3_5x5, kernel_size=(5,5), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(conv3)

  conv4 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(prev_layer)
  conv4 = Conv2D(filters=filters_4_1x1, kernel_size=(1,1), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(conv4)

  return concatenate([conv1, conv2, conv3, conv4], axis=3, name=name)

In [8]:
input_shape = (64, 64, 3)


inputs = tf.keras.Input(shape=input_shape)
x=  Rescaling(1.0 / 255)(inputs)
x = Conv2D(filters=16, kernel_size=(7,7), strides=(2,2), padding='same', activation='relu', name='conv1_7x7/2', bias_initializer=tf.keras.initializers.Constant(value=0.2))(x)

x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='maxpool2_3x3/2')(x)
x = Conv2D(filters=16, kernel_size=(1,1), padding='same', activation='relu', name='conv2_1x1/1', bias_initializer=tf.keras.initializers.Constant(value=0.2))(x)
x = Conv2D(filters=48, kernel_size=(3,3), padding='same', activation='relu', name='conv2_3x3/1', bias_initializer=tf.keras.initializers.Constant(value=0.2))(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='maxpool3_3x3/2')(x)

x = inception_layer(x, filters_1_1x1=16, filters_2_1x1=24, filters_2_3x3=32, filters_3_1x1=4, filters_3_5x5=8, filters_4_1x1=8, name='inception3_1')
x = inception_layer(x, filters_1_1x1=32, filters_2_1x1=32, filters_2_3x3=48, filters_3_1x1=8, filters_3_5x5=24, filters_4_1x1=16, name='inception3_2')
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='maxpool4_3x3/2')(x)

x = inception_layer(x, filters_1_1x1=48, filters_2_1x1=24, filters_2_3x3=52, filters_3_1x1=4, filters_3_5x5=12, filters_4_1x1=16, name='inception4_1')
## first output
x1 = AveragePooling2D(pool_size=(5, 5), strides=(3,3), padding='same', name='out1_avgpool_5x5/3')(x)
x1 = Conv2D(filters=32, kernel_size=(1,1), padding='same', activation='relu', name='out1_conv_1x1/1', bias_initializer=tf.keras.initializers.Constant(value=0.2))(x1)
x1 = Flatten()(x1)
x1 = Dense(64, activation='relu', name='out1_dense1')(x1)
x1 = Dropout(0.7, name='out1_dropout')(x1)
x1 = Dense(1, activation='sigmoid', name='out1')(x1)

x = inception_layer(x, filters_1_1x1=40, filters_2_1x1=28, filters_2_3x3=56, filters_3_1x1=6, filters_3_5x5=16, filters_4_1x1=16, name='inception4_2')
x = inception_layer(x, filters_1_1x1=32, filters_2_1x1=32, filters_2_3x3=64, filters_3_1x1=6, filters_3_5x5=16, filters_4_1x1=16, name='inception4_3')
x = inception_layer(x, filters_1_1x1=28, filters_2_1x1=36, filters_2_3x3=72, filters_3_1x1=8, filters_3_5x5=16, filters_4_1x1=16, name='inception4_4')
## second output
x2 = AveragePooling2D(pool_size=(5, 5), strides=(3,3), padding='same', name='out2_avgpool_5x5/3')(x)
x2 = Conv2D(filters=32, kernel_size=(1,1), padding='same', activation='relu', name='out2_conv_1x1/1', bias_initializer=tf.keras.initializers.Constant(value=0.2))(x2)
x2 = Flatten()(x2)
x2 = Dense(64, activation='relu', name='out2_dense1')(x2)
x2 = Dropout(0.7, name='out2_dropout')(x2)
x2 = Dense(1, activation='sigmoid', name='out2')(x2)

x = inception_layer(x, filters_1_1x1=64, filters_2_1x1=40, filters_2_3x3=80, filters_3_1x1=8, filters_3_5x5=32, filters_4_1x1=32, name='inception4_5')
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='maxpool5_3x3/2')(x)

x = inception_layer(x, filters_1_1x1=64, filters_2_1x1=40, filters_2_3x3=80, filters_3_1x1=8, filters_3_5x5=32, filters_4_1x1=32, name='inception5_1')
x = inception_layer(x, filters_1_1x1=96, filters_2_1x1=48, filters_2_3x3=96, filters_3_1x1=12, filters_3_5x5=32, filters_4_1x1=32, name='inception5_2')
## third output
x = AveragePooling2D(pool_size=(7, 7), strides=(1,1), padding='same', name='out3_avgpool_7x7/1')(x)
x = Flatten()(x)
x = Dropout(0.7, name='out3_dropout')(x)
x = Dense(1, activation='sigmoid', name='out3')(x)

model = tf.keras.Model(inputs=inputs, outputs=[x, x1, x2], name='Inception-v1')
model.summary()


Model: "Inception-v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 64, 64, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_7x7/2 (Conv2D)           (None, 32, 32, 16)   2368        ['rescaling[0][0]']              
                                                                                                  
 maxpool2_3x3/2 (MaxPooling2D)  (None, 16, 16, 16)   0           ['conv1_7x7/2[0][0]']            
                                                                                       

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard, LearningRateScheduler

In [12]:
initial_lr = 0.01
def scheduler(epoch, lr):
  drop = 0.96
  epochs_drop = 8
  lrate = lr * math.pow(drop, math.floor((1 + epoch)/epochs_drop))
  return lrate
loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=initial_lr, momentum=0.9, nesterov=False, name='SGD')
callbacks = [
          LearningRateScheduler(scheduler),
          #Luu bo trong so weight    
          ModelCheckpoint("files/model.h5"),
          #Theo doi loss function va giam LR
          #factor gia tri learning rate se giam 
          #patience: sau so chu ky ma LR se giam
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
          #Truyen lai ket qua vao CSV
          CSVLogger("files/data.csv"),
          TensorBoard(),
          #Theo doi loss function
          #patience: sau so chu ky neu khong cai thien thi dung
          #restore_best_weights: lua chon khoi phuc tu epoch co ket qua tot nhat
          EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
]

In [13]:
model.compile(optimizer=optimizer,
              loss=[loss, loss, loss], loss_weights=[1, 0.3, 0.3],
              metrics=['accuracy'])

# Fit model (training)
history = model.fit(train_ds, steps_per_epoch=len(train_ds),
    validation_data=val_ds, validation_steps=len(val_ds), epochs=40, verbose=1,callbacks=callbacks)

Epoch 1/40
544/544 [==============================] - 37s 61ms/step - loss: 0.0435 - out3_loss: 0.0232 - out1_loss: 0.0329 - out2_loss: 0.0348 - out3_accuracy: 0.9917 - out1_accuracy: 0.9890 - out2_accuracy: 0.9900 - val_loss: 0.0563 - val_out3_loss: 0.0336 - val_out1_loss: 0.0384 - val_out2_loss: 0.0372 - val_out3_accuracy: 0.9885 - val_out1_accuracy: 0.9862 - val_out2_accuracy: 0.9851 - lr: 0.0100
Epoch 2/40
544/544 [==============================] - 33s 60ms/step - loss: 0.0370 - out3_loss: 0.0202 - out1_loss: 0.0263 - out2_loss: 0.0297 - out3_accuracy: 0.9927 - out1_accuracy: 0.9909 - out2_accuracy: 0.9918 - val_loss: 0.0981 - val_out3_loss: 0.0598 - val_out1_loss: 0.0616 - val_out2_loss: 0.0660 - val_out3_accuracy: 0.9784 - val_out1_accuracy: 0.9828 - val_out2_accuracy: 0.9821 - lr: 0.0100
Epoch 3/40
544/544 [==============================] - 33s 59ms/step - loss: 0.0358 - out3_loss: 0.0188 - out1_loss: 0.0300 - out2_loss: 0.0265 - out3_accuracy: 0.9936 - out1_accuracy: 0.9913 - o